In [28]:
import sys 
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.pose import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.protocols.minimization_packing import *
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose, glycosylate_pose_by_file
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.protocols.rosetta_scripts import *

## Personal Imports
import basePeptidePrep
import addBaseSugarAndEnzyme

## Pyrosetta Initialization
pyrosetta.init('-include_sugars -write_pdb_link_records -ex1 -ex2 -alternate_3_letter_codes pdb_sugar -auto_detect_glycan_connections -min_bond_length 1.1 -max_bond_length 1.5 -ignore_zero_occupancy false -ignore_unrecognized_res false')

## Command Line Arguments 
'''
Command Line Arguments:
# For the base peptide 
- Sequence of the base peptide (2-3 AA long peptides are ideal) : base_seq
- Position of the sequence containing THR that will be glycosylated : base_pos
- the sugar that will be glycosylated on the base peptide : base_sugar

# input pdb files
- The enzyme with donor peptide 

- The constraint file

'''
base_seq = "STP"
base_position = 2
base_sugar = None #"core1"
input_enzyme_file = "/home/souvadra/myGitFolders/Glycosyltransferase/side_project/input_substrates/2ffu-monomer-alpha-GalNAc-metal_removed.pdb" # ra/myGitFolders/Glycosyltransferase/glycan_sampler_pipeline/output_3OTK-closed-S217C/3OTK-closed-monomer-alpha-GlcNAc-S217C_0005.pdb"
constraints_file = "ppGalNAC_T2_constraints_file.cst" #"constraints_file.cst"
#constraints_file = "trialConstraint.cst"
## Prepare the base peptide 
base_pose = basePeptidePrep.basePeptideBuild(base_seq, base_position, base_sugar)

## Addition of peptide and sugar motif 
enzyme_pose = pose_from_pdb(input_enzyme_file)

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.40+release.f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 2020-10-02T11:47:11] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r267 2020.40+release.f01fa77 f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 http://www.pyrosetta.org 2020-10-02T11:47:11
core.init: {0} command: PyRosetta -include_sugars -write_pdb_link_records -ex1 -ex2 -alternate_3_letter_codes pdb_sugar -auto_detect_glycan_connections -min_bond_length 1.1 -max_bond_length 1.5 -ignore_zero_occupancy false -ignore_unrecognized_res false -database /home/souvadra/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator:

In [29]:
print(enzyme_pose.fold_tree())


FOLD_TREE  EDGE 1 494 -1  EDGE 1 495 1 


In [30]:
def mergePoses(base_pose, enzyme_pose):
    """merge the base pose into enzyme pose without making any covalent bond"""
    anchor_point = int(enzyme_pose.total_residue()) - 2 # just to make sure that the UDP-sugar remains the last residues in the whole pose
    enzyme_pose.append_pose_by_jump(base_pose, anchor_point)

mergePoses(base_pose, enzyme_pose)

core.kinematics.FoldTree: {0} [ WARNING ] insert_fold_tree_by_jump: anchor_pos is not a vertex of the tree!! anchor_pos= 493
core.kinematics.FoldTree: {0} [ WARNING ] *this= FOLD_TREE  EDGE 1 494 -1  EDGE 1 495 1
core.kinematics.FoldTree: {0} [ WARNING ] Adding anchor_pos as a new vertex!


In [31]:
residue1 = enzyme_pose.residue(495).xyz("3OPB")
residue2 = enzyme_pose.residue(497).xyz("CB")
print((residue1-residue2).norm())

54.70077725166518


In [32]:
print(enzyme_pose.residue(497))

Residue 497: THR (THR, T):
Base: THR
 Properties: POLYMER PROTEIN CANONICAL_AA SC_ORBITALS POLAR BETA_BRANCHED_SIDECHAIN METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   OG1  CG2  HB   HG1 1HG2 2HG2 3HG2
Atom Coordinates:
   N  : 3.33248, 1.53597, 1.45999e-16
   CA : 3.98758, 2.83851, 1.11973e-17
   C  : 5.50383, 2.69251, 3.98699e-16
   O  : 6.02991, 1.57957, 1.02723e-15
   CB : 3.54895, 3.67965, 1.21303
   OG1: 2.61316, 2.93341, 2.002
   CG2: 2.89987, 4.97701, 0.755875
   H  : 3.89945, 0.700123, 4.63798e-16
   HA : 3.70087, 3.36994, -0.907762
   HB : 4.41811, 3.91188, 1.82811
   HG1: 2.46818, 2.07442, 1.59792
  1HG2: 2.59668, 5.55853, 1.62629
  2HG2: 3.61277, 5.55205, 0.165202
  3HG2: 2.02452, 4.75122, 0.14823
Mirrored relative to coordinates in ResidueType: FALSE



In [33]:
print(enzyme_pose.residue(495))

Residue 495: UDP:non-conjugated (UDP, Z):
Base: UDP
 Properties: POLYMER NUCLEOTIDE_DIPHOSPHATE LOWER_TERMINUS UPPER_TERMINUS POLAR AROMATIC CYCLIC
 Variant types: UPPER_TERMINUS_VARIANT LOWER_TERMINUS_VARIANT
 Main-chain atoms:  C1'  C2'  C3'  C4'  C5'  O5'  PA  3OPA  PB  3OPB
 Backbone atoms:    C1'  C2'  C3'  C4'  C5'  O5'  PA  3OPA  PB  3OPB  O4' 1OPA 2OPA 1OPB 2OPB VO4' VC1'  H1'  H2'  H3'  H4' 1H5' 2H5'
 Ring atoms:    C1'  C2'  C3'  C4'  O4'
 Side-chain atoms:  N1   C2   O2   N3   C4   O4   C5   C6   O2'  O3'  H3   H5   H6  HO2' HO3'
Ring Conformer: 2T1 (twist): C-P parameters (q, phi, theta): 0.4, 54; nu angles (degrees): 60, -60, 30, 30, -60
  N1 : equatorial
  O2': equatorial
  O3': axial
  C5': neither axial nor equatorial
Atom Coordinates:
   C1': 15.188, 2.766, -55.535
   C2': 15.128, 2.484, -54.047
   C3': 16.305, 3.325, -53.631
   C4': 17.315, 3.048, -54.604
   C5': 18.474, 2.044, -54.301
   O5': 17.867, 0.816, -53.93
   PA : 18.206, 0.242, -52.529
  3OPA: 19.617, -0.459

In [34]:
print(enzyme_pose.residue(275))

Residue 275: LEU (LEU, L):
Base: LEU
 Properties: POLYMER PROTEIN CANONICAL_AA HYDROPHOBIC ALIPHATIC METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Side-chain atoms:  CB   CG   CD1  CD2 1HB  2HB   HG  1HD1 2HD1 3HD1 1HD2 2HD2 3HD2
Atom Coordinates:
   N  : 36.188, 7.978, -62.437
   CA : 35.312, 7.9, -61.272
   C  : 36.107, 8.372, -60.064
   O  : 37.261, 7.981, -59.889
   CB : 34.851, 6.46, -61.029
   CG : 34.123, 5.717, -62.15
   CD1: 33.807, 4.304, -61.683
   CD2: 32.84, 6.457, -62.521
   H  : 37.0187, 7.40426, -62.4672
   HA : 34.433, 8.51593, -61.4586
  1HB : 35.7212, 5.85356, -60.7832
  2HB : 34.1746, 6.45105, -60.1744
   HG : 34.7686, 5.65799, -63.0267
  1HD1: 33.2877, 3.76616, -62.4763
  2HD1: 34.7344, 3.78488, -61.4408
  3HD1: 33.1725, 4.3467, -60.7985
  1HD2: 32.3287, 5.92054, -63.3209
  2HD2: 32.1884, 6.51531, -61.6489
  3HD2: 33.0854, 7.46405, -62.8586
Mirrored relative to coordinates in ResidueTy

In [35]:
print(enzyme_pose.residue(537))

RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/pose/Pose.cc:951
[ ERROR ] UtilityExitException
ERROR: Pose::residue( Size const seqpos ): variable seqpos is out of range!



In [38]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.pose import *
from pyrosetta.rosetta.core.scoring import *
from pyrosetta.rosetta.protocols.minimization_packing import *
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose, glycosylate_pose_by_file
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.protocols.rosetta_scripts import *

from pyrosetta.rosetta.core.scoring.constraints import *
from pyrosetta.rosetta.core.select.movemap import MoveMapFactory

""" This program takes the base sugar pose and the pose of the enzyme with donor moiety
     and the constraints and make sure the they obey the constraint in subsequent relax
       procedure as a procedure to bypass manual overlaying using PyMOL. """
init("-constraints:cst_fa_file " + constraints_file)

sfxn = pyrosetta.create_score_function("ref2015_cart.wts") #get_fa_scorefxn()
initial_energy = sfxn(enzyme_pose)
'''
sfxn.set_weight(atom_pair_constraint, 1.0)
add_fa_constraints_from_cmdline(enzyme_pose, sfxn)
'''
print(sfxn.show(enzyme_pose))

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.40+release.f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 2020-10-02T11:47:11] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r267 2020.40+release.f01fa77 f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 http://www.pyrosetta.org 2020-10-02T11:47:11
core.init: {0} command: PyRosetta -constraints:cst_fa_file ppGalNAC_T2_constraints_file.cst -database /home/souvadra/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1592909475 seed_offset=0 real_seed=-1592909475 thread_index=0
basic.random.init_random_generator: {0} Ran

In [6]:
print(enzyme_pose.sequence())

KVRWPDFNQEAYVGGTMVRSGQDPYARNKFNQVESDKLRMDRAIPDTRHDQCQRKQWRVDLPATSVVITFHNEARSALLRTVVSVLKKSPPHLIKEIILVDDYSNDPEDGALLGKIEKVRVLRNDRREGLMRSRVRGADAAQAKVLTFLDSHCECNEHWLEPLLERVAEDRTRVVSPIIDVINMDNFQYVGASADLKGGFDWNLVFKWDYMTPEQRRSRQGNPVAPIKTPMIAGGLFVMDKFYFEELGKYDMMMDVWGGENLEISFRVWQCGGSLEIIPCSRVGHVFRKQHPYTFPGGSGTVFARNTRRAAEVWMDEYKNFYYAAVPSARNVPYGNIQSRLELRKKLSCKPFKWYLENVYPELRVPDHQDIAFGALQQGTNCLDTLGHFADGVVGVYECHNAGGNQEWALTKEKSVKHMDLCLTVVDRAPGSLIKLQGCRENDSRQKWEQIEGNSKLRHVGSNLCLDSRTAKSGGLSVEVCGPALSQQWKFTLNZSTP


In [7]:
chain_begin_list = []
chain_end_list = []
chain_begin_list.append(int(enzyme_pose.chain_begin(1)))
chain_end_list.append(int(enzyme_pose.chain_end(1)))
while (chain_end_list[-1] < int(enzyme_pose.total_residue())):
    chain_num = int(enzyme_pose.chain(chain_end_list[-1]+1))
    chain_begin_list.append(int(enzyme_pose.chain_begin(chain_num)))
    chain_end_list.append(int(enzyme_pose.chain_end(chain_num)))

In [8]:
print(chain_begin_list)
print(chain_end_list)
total_chains = len(chain_begin_list)
print(total_chains)

[1, 495, 496]
[494, 495, 498]
3


In [9]:
#AA_set = set()
#for i in range(chain_begin_list[3],chain_end_list[3]+1):
#    name = str(enzyme_pose.residue(i).name())
#    print(name)

In [10]:
AA_set = {'ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL'}

In [11]:
# Criteria 1: Enzyme generally the largest chain of this reaction 
chain_length_list = []
for i in range(0,len(chain_begin_list)):
    chain_length_list.append(chain_end_list[i]-chain_begin_list[i])
largest_chain_length = max(chain_length_list)
for i in range(0,len(chain_length_list)):
    if chain_length_list[i] == largest_chain_length:
        enzyme_chain = i+1;
print(enzyme_chain)
# Criteria 2: Acceptor peptide will be the only another chain without any sugar in it
for i in range(0,len(chain_begin_list)):
    if (i+1 != enzyme_chain):
        assumption = True
        for j in range(chain_begin_list[i],chain_end_list[i]+1):
            name = str(enzyme_pose.residue(j).name())
            name = name[0:3]
            if name == 'UDP': donor_chain = i+1
            if name not in AA_set:
                assumption = False
        if assumption == True:
            acceptor_chain = i+1
            print(acceptor_chain)
print(donor_chain)

1
3
2


In [12]:
#print(enzyme_pose.residue(370))

In [24]:
reference = enzyme_pose.residue(chain_begin_list[donor_chain-1]).xyz("3OPB")
distance_lowest = float('inf')
pivot_residue = float('inf')
for i in range(chain_begin_list[enzyme_chain-1],chain_end_list[enzyme_chain-1]+1):
    #if i == 288 or i == 287: continue
    residue1 = enzyme_pose.residue(i).xyz("CA")
    distance_curr = (residue1-reference).norm()
    if distance_curr < distance_lowest:
        distance_lowest = distance_curr
        pivot_residue = i
print(distance_lowest)
print(pivot_residue)

5.315526878870988
288


In [14]:
print(enzyme_chain)
print(donor_chain)
print(acceptor_chain)
#print(enzyme_pose.chain(373))

1
2
3


In [15]:
## Change the FoldTree to more Docking friendly option as suggested by Pooja 
first_residue_enzyme = int(enzyme_pose.chain_begin(enzyme_chain)) 
last_residue_enzyme = int(enzyme_pose.chain_end(enzyme_chain))

anchor_residue_enzyme = pivot_residue # 130 ## Right now, I am keeping it as a user input, I'll change it later

first_residue_substrate = int(enzyme_pose.chain_begin(acceptor_chain))
last_residue_substrate = int(enzyme_pose.chain_end(acceptor_chain))
anchor_residue_substrate = int((first_residue_substrate+last_residue_substrate)/2)
first_donor = int(enzyme_pose.chain_begin(donor_chain))
last_donor = int(enzyme_pose.chain_end(donor_chain))

ft_docking = pyrosetta.FoldTree()
ft_docking.add_edge(anchor_residue_enzyme,1,-1)
ft_docking.add_edge(anchor_residue_enzyme,last_residue_enzyme,-1)

ft_docking.add_edge(anchor_residue_enzyme, anchor_residue_substrate, 1) 
ft_docking.add_edge(anchor_residue_enzyme, first_donor, 2)
            
ft_docking.add_edge(anchor_residue_substrate, first_residue_substrate, -1) # downstream base peptide 
ft_docking.add_edge(anchor_residue_substrate, last_residue_substrate, -1) # upstream base peptide 
ft_docking.add_edge(first_donor,last_donor,-1) # the donor UDP-GlcNAc backbone

chemical_edges = enzyme_pose.fold_tree().get_chemical_edges() # connections to the glycan residues from anchor out 
for chemical in range(1, len(chemical_edges) + 1):
    ft_docking.add_edge(chemical_edges[chemical].start(), chemical_edges[chemical].stop(), -2)
    sugar_start_res = chemical_edges[chemical].stop()
    sugar_chain = int(enzyme_pose.chain(sugar_start_res))
    sugar_chain_start = int(enzyme_pose.chain_begin(sugar_chain))
    sugar_chain_end = int(enzyme_pose.chain_end(sugar_chain))
    ft_docking.add_edge(sugar_chain_start,sugar_chain_end,-1)
    
print(enzyme_pose.fold_tree())
enzyme_pose.fold_tree(ft_docking)
print(enzyme_pose.fold_tree())
    
    ## setting the movemap 
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(False)
mm.set_chi(False)
mm.set_jump(True)

referencePose = None #pose_from_pdb("/home/souvadra/myGitFolders/Glycosyltransferase/Acceptor-Donor-Enzyme/GlcNAc-added-before-GalBGalNAc/3OTK-closed-monomer-alpha-GlcNAc_2GAM-GalBGalNAc.pdb")  
RMSD_list = []
score_list = []

FOLD_TREE  EDGE 1 495 1  EDGE 1 493 -1  EDGE 493 494 -1  EDGE 493 496 2  EDGE 496 498 -1 
FOLD_TREE  EDGE 288 1 -1  EDGE 288 494 -1  EDGE 288 497 1  EDGE 288 495 2  EDGE 497 496 -1  EDGE 497 498 -1 


In [ ]:
#pose_trial = pose_from_pdb("merging_result0.pdb")
#enzyme_pose.dump_pdb("trial_enzyme_combined.pdb")

In [ ]:
#trial_pose = pose_from_pdb("trial_enzyme_combined.pdb")

In [ ]:
    for trial_number in range(0,10):
        ## Use Rigid Body Mover to bring the base peptide close to the enzyme
        curr_enzyme_pose = enzyme_pose.clone()
        rigid_body = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body.add_jump(1)
        rigid_body.rot_magnitude(10)
        rigid_body.trans_magnitude(5.0)
        
        first_pose = curr_enzyme_pose.clone()
        num_iter = 0
        while (sfxn(curr_enzyme_pose) > 0 ):
            num_iter += 1
            rigid_body.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose))
            else:
                first_pose = curr_enzyme_pose.clone()
            if (num_iter > 500):
                break
            #print(sfxn(enzyme_pose))
        sfxn.show(curr_enzyme_pose)
        print(curr_enzyme_pose.fold_tree())

        rigid_body2 = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body2.add_jump(1)
        rigid_body2.rot_magnitude(0)
        rigid_body2.trans_magnitude(0.1)
        
        rigid_body_rot = pyrosetta.rosetta.protocols.rigid.RigidBodyPerturbNoCenterMover()
        rigid_body_rot.add_jump(1)
        rigid_body_rot.rot_magnitude(1)
        rigid_body_rot.trans_magnitude(0)
        
        first_pose = curr_enzyme_pose.clone()
        num_iter = 0
        while (sfxn(curr_enzyme_pose) > (initial_energy) ):
            num_iter += 1
            rigid_body_rot.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose), "  rotation-iteration ")    
            rigid_body2.apply(first_pose)
            if (sfxn(first_pose) < sfxn(curr_enzyme_pose)):
                curr_enzyme_pose = first_pose.clone()
                print(sfxn(curr_enzyme_pose), "  translation-iteration ")
            else:
                first_pose = curr_enzyme_pose.clone()
            print(num_iter)
            if (num_iter > 500):
                break
            #print(sfxn(enzyme_pose))
        score_list.append(sfxn(curr_enzyme_pose))
        RMSD_list.append(CA_rmsd(curr_enzyme_pose,referencePose))
        dumping_name = "merging_result" + str(trial_number) + ".pdb"
        curr_enzyme_pose.dump_pdb(dumping_name)

In [ ]:
score_list =  [773.4666756904846, -318.93840946168575, 1697.5004297047583, -412.9292382544952, -143.01147681384555, -400.8640686228233, -22.162164032408327, -405.205297699855, -411.6214919105398, -410.7368805728968, -411.9609342790661, -411.5920211360774, 1463.2421233287737, -410.52774273393493, -411.6392010088241, -411.3877064505025, -405.3111815912032, -405.6641760660225, -407.50873752734446, -410.51901301893963]
RMSD_list = [1.6200072765350342, 1.3890552520751953, 2.1524126529693604, 0.962411642074585, 0.8145793080329895, 0.8073674440383911, 1.0122199058532715, 0.8093990683555603, 0.6696670651435852, 0.6450819969177246, 1.0149338245391846, 0.6716275215148926, 2.217975616455078, 0.6443003416061401, 0.6652771234512329, 0.6686609983444214, 0.9484436511993408, 0.6377775073051453, 0.9520885944366455, 0.6440408825874329]

In [ ]:
import matplotlib.pylab as plt
plt.scatter(RMSD_list,score_list)
plt.xlabel("RMSD-value")
plt.ylabel("REU score")

In [ ]:
score_list

In [ ]:
CA_rmsd(enzyme_pose,referencePose)

In [ ]:
initial_energy